In [ ]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-1000)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [ ]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

mypf = pd.read_csv('/content/drive/My Drive/data/stocks/Nifty100Scrips.csv')

#mypf = mypf[mypf['Nifty100'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# star stocks & my stocks
star_stocks = list(mypf[mypf['StarStock'] == 1]['Stock'].values)
my_stocks = list(mypf[mypf['InPortfolio'] == 1]['Symbol'].values)

In [ ]:
# update Nifty features
'''
stock = 'INFY'
column = 'Conviction'
value = 'H'
nifty100_df.loc[nifty100_df['Symbol'] == stock, column] = value

mypf.to_csv('/content/drive/My Drive/data/stocks/Nifty100Scrips.csv',index=False)
'''
mypf['Conviction'].value_counts()

M    76
L    63
H    37
Name: Conviction, dtype: int64

In [ ]:
# star stocks
mypf['StarStock'].value_counts()

0    98
1    78
Name: StarStock, dtype: int64

In [ ]:
# v40/v40n stocks
mypf['Criteria'].value_counts()

V40N    39
V40     38
2T3Y     8
V200     6
SS       5
TBD      2
Name: Criteria, dtype: int64

In [ ]:
# latest quarter
mypf['LatestQtr'].value_counts()

1    150
0     26
Name: LatestQtr, dtype: int64

In [ ]:
# star + v40/v40n stocks
mypf[(mypf['Criteria'] == 'V40') | (mypf['Criteria'] == 'V40N')]['StarStock'].value_counts()

0    39
1    38
Name: StarStock, dtype: int64

In [ ]:
# stocks in portfolio
mypf['InPortfolio'].value_counts()

0    143
1     33
Name: InPortfolio, dtype: int64

In [ ]:
# star stocks in portfolio
mypf[(mypf['InPortfolio'] == 1)]['StarStock'].value_counts()

1    20
0    13
Name: StarStock, dtype: int64

In [ ]:
# criteria of portfolio stocks
mypf[(mypf['InPortfolio'] == 1)]['Criteria'].value_counts()

V40     10
2T3Y     8
V40N     7
SS       5
TBD      2
V200     1
Name: Criteria, dtype: int64

In [ ]:
# strategies in portfolio stocks
mypf[(mypf['InPortfolio'] == 1)]['Strategy'].value_counts()

ATH     21
2T3Y    12
Name: Strategy, dtype: int64

In [ ]:
# import libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib 
yfin.pdr_override()
import pandas_ta as ta

In [ ]:
# fetch current PE
import requests
from bs4 import BeautifulSoup

def get_current_pe(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  stocks = ['INDIGOPNTS', 'IRCTC', 'GILLETTE', 'PAGEIND']

  if stock_name in stocks:
    actual_link = link_blueprint + stock_name.upper()  
  else:
    actual_link = link_blueprint + stock_name.upper() + "/consolidated/"

  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")        
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:
    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
  except ValueError as ve:
    current_pe = 1000 

  return current_pe


In [ ]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20 
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'
    
    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = yfin.Ticker(stock_symbol).history(period='3y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    #print(stock_df)

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close'].rolling(window = short_window, min_periods = 1).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close'].rolling(window = mid_window, min_periods = 1).mean(),0)     

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df['200_SMA_Lag10'] = stock_df['200_SMA'].shift(10)
    stock_df['Slope%_200'] = round((stock_df['200_SMA'] - stock_df['200_SMA_Lag10'])*100/stock_df['200_SMA'],2)
    stock_df['50_SMA_Lag5'] = stock_df['50_SMA'].shift(5)
    stock_df['Slope%_50'] = round((stock_df['50_SMA'] - stock_df['50_SMA_Lag5'])*100/stock_df['50_SMA'],2)
    
    #stock_df.ta.rsi(append=True)
    #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
    stock_df['Close'] = round(stock_df['Close'],0)    
    #stock_df['RSI'] = round(stock_df['RSI'],0) 
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df['Min'] = round(min(stock_df['Close']),0)   
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Upside%ATH'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume','20_SMA','50_SMA','200_SMA','200_SMA_Lag10','50_SMA_Lag5'
                   ], axis=1, inplace=True)   
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    if stock_id in my_stocks:
      current_pe = get_current_pe(stock_id)
    else:
      current_pe = 1000

    stock_id = stock_id.upper() + '.NS'    
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Shares'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]
    tmp['Med_PE'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['MedianPE'].values[0]    
    tmp['Curr_PE'] = current_pe
    tmp['BizOutlook'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['BizOutlook'].values[0] 
    tmp['BizDurability'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['BizDurability'].values[0]   
    tmp['Conviction'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Conviction'].values[0]    
    tmp['LatestQtr'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['LatestQtr'].values[0]  
    tmp['StarStock'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['StarStock'].values[0]       
    tmp['Criteria'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Criteria'].values[0] 
    tmp['Strategy'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Strategy'].values[0]
    df_prec_dev = df_prec_dev.append(tmp)

In [ ]:
# nifty in last 5 weeks
stock_prec_dev('^NSEI')[-21::5]

,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH
Date,,,,,,,,,
2023-04-27 00:00:00+05:30,17915.0,^NSEI,1.53,0.57,-0.03,18812.0,10806.0,11.20,5.0
2023-05-05 00:00:00+05:30,18069.0,^NSEI,2.11,0.56,0.12,18812.0,10806.0,9.28,4.0
2023-05-12 00:00:00+05:30,18315.0,^NSEI,3.18,0.59,0.31,18812.0,10806.0,6.21,3.0
2023-05-19 00:00:00+05:30,18203.0,^NSEI,2.32,0.54,0.48,18812.0,10806.0,7.61,3.0
2023-05-26 00:00:00+05:30,18499.0,^NSEI,3.79,0.41,0.42,18812.0,10806.0,3.91,2.0


In [ ]:
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)
df_prec_dev.drop(['Date'], axis=1, inplace=True)

mypf['Stock'] = mypf['Symbol'] + '.NS'
#df_prec_dev.drop(['Med_PE'], axis=1, inplace=True)

In [ ]:
df_prec_dev['Discount%PE'] = round((df_prec_dev['Med_PE']-df_prec_dev['Curr_PE'])*100/(df_prec_dev['Med_PE']),0)
#df_prec_dev['Upside%Estimate'] = round((df_prec_dev['Max']-df_prec_dev['Avg_Cost'])*100/(df_prec_dev['Avg_Cost']),0)
df_prec_dev = df_prec_dev.set_index('Stock')

In [ ]:
# portfolio stocks
cols = ['Close',	'Dev%_200',	'Slope%_200',	'Slope%_50', 'Discount%',	'Upside%ATH',	'Avg_Cost',	'Med_PE',	'Curr_PE',	'Conviction', 'Discount%PE', 'LatestQtr', 'StarStock', 'Criteria', 'Strategy']
tmp_df = df_prec_dev[cols]
tmp_df = tmp_df[tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 33


,Close,Dev%_200,Slope%_200,Slope%_50,Discount%,Upside%ATH,Avg_Cost,Med_PE,Curr_PE,Conviction,Discount%PE,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
GLAND.NS,929.0,-44.49,-3.46,-2.62,98.98,365.0,1354.78,45.2,18.60,M,59.0,1,0,2T3Y,2T3Y
VALIANTORG.NS,554.0,-3.19,-1.40,2.34,88.17,234.0,481.73,24.0,17.70,M,26.0,0,0,2T3Y,2T3Y
VAIBHAVGBL.NS,298.0,-7.10,0.00,-0.66,96.12,224.0,320.64,35.4,46.90,M,-32.0,1,0,2T3Y,2T3Y
LUXIND.NS,1431.0,-9.06,-1.34,1.05,91.16,218.0,1499.87,21.4,23.80,M,-11.0,0,0,V40N,2T3Y
TEAMLEASE.NS,2241.0,-14.82,-2.47,-1.31,93.45,136.0,2239.86,NaN,33.80,M,NaN,1,1,V40N,2T3Y
AAVAS.NS,1368.0,-27.90,-2.42,-2.81,97.44,134.0,1405.00,NaN,25.20,M,NaN,1,1,2T3Y,2T3Y
INDIGOPNTS.NS,1507.0,14.81,0.00,3.34,75.42,107.0,1312.07,104.9,54.40,M,48.0,1,1,V40N,2T3Y
JUBLFOOD.NS,484.0,-7.17,-0.96,0.44,86.39,87.0,468.21,90.5,90.40,H,0.0,1,1,SS,ATH
IRCTC.NS,624.0,-5.69,0.15,0.17,59.54,87.0,629.08,77.4,53.00,H,32.0,0,1,SS,ATH


In [ ]:
# top 10 from portfolio for SIP
tmp_df.sort_values(by = 'Upside%ATH', ascending=False).head(7)

,Close,Dev%_200,Slope%_200,Slope%_50,Discount%,Upside%ATH,Avg_Cost,Med_PE,Curr_PE,Conviction,Discount%PE,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
GLAND.NS,929.0,-44.49,-3.46,-2.62,98.98,365.0,1354.78,45.2,18.6,M,59.0,1,0,2T3Y,2T3Y
VALIANTORG.NS,554.0,-3.19,-1.40,2.34,88.17,234.0,481.73,24.0,17.7,M,26.0,0,0,2T3Y,2T3Y
VAIBHAVGBL.NS,298.0,-7.10,0.00,-0.66,96.12,224.0,320.64,35.4,46.9,M,-32.0,1,0,2T3Y,2T3Y
LUXIND.NS,1431.0,-9.06,-1.34,1.05,91.16,218.0,1499.87,21.4,23.8,M,-11.0,0,0,V40N,2T3Y
TEAMLEASE.NS,2241.0,-14.82,-2.47,-1.31,93.45,136.0,2239.86,NaN,33.8,M,NaN,1,1,V40N,2T3Y
AAVAS.NS,1368.0,-27.90,-2.42,-2.81,97.44,134.0,1405.00,NaN,25.2,M,NaN,1,1,2T3Y,2T3Y
INDIGOPNTS.NS,1507.0,14.81,0.00,3.34,75.42,107.0,1312.07,104.9,54.4,M,48.0,1,1,V40N,2T3Y


In [ ]:
# top 5 from portfolio to book profit
tmp_df.sort_values(by = 'Upside%ATH', ascending=True).head(7)

,Close,Dev%_200,Slope%_200,Slope%_50,Discount%,Upside%ATH,Avg_Cost,Med_PE,Curr_PE,Conviction,Discount%PE,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
EICHERMOT.NS,3690.0,10.52,0.84,1.45,8.67,4.0,3396.72,44.7,34.6,H,23.0,1,1,V40N,ATH
TATACONSUM.NS,790.0,4.46,-0.13,0.96,18.45,10.0,758.64,68.8,68.0,H,1.0,1,1,SS,ATH
RELIANCE.NS,2506.0,1.40,-0.04,0.38,31.12,12.0,2464.49,29.1,25.4,H,13.0,1,1,V40,ATH
PIDILITIND.NS,2586.0,1.58,0.31,0.87,20.87,12.0,2442.09,94.6,103.0,H,-9.0,1,1,V40,ATH
ASIANPAINT.NS,3128.0,1.97,-0.13,0.92,25.84,14.0,3048.30,87.9,72.4,H,18.0,1,1,V40,ATH
BAJFINANCE.NS,6904.0,4.57,0.11,1.34,20.40,14.0,6365.23,52.6,36.3,H,31.0,1,1,V40,ATH
PNB.NS,50.0,7.13,2.13,0.00,29.41,20.0,50.60,NaN,16.6,L,NaN,1,1,SS,ATH


In [ ]:
df_prec_dev.drop([ 'Avg_Cost', 'Shares', 'Med_PE', 'Curr_PE', 'Discount%PE'], axis=1, inplace=True)

In [ ]:
# star + v40/v40n stocks (for upside)

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 9


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
FINEORG.NS,4473.0,-19.07,-0.72,-0.11,7196.0,2199.0,54.49,61.0,NaN,NaN,M,1,1,V40N,NaN
DIXON.NS,3661.0,0.42,-0.77,1.82,5750.0,1618.0,50.56,57.0,NaN,NaN,M,1,1,V40N,NaN
ANGELONE.NS,1278.0,-2.98,-0.38,1.00,1894.0,208.0,36.54,48.0,NaN,NaN,M,1,1,V40N,NaN
BAYERCROP.NS,4125.0,-9.48,-1.12,-0.10,6002.0,3970.0,92.37,46.0,NaN,NaN,M,1,1,V40N,NaN
BAJAJFINSV.NS,1439.0,-4.88,0.20,0.52,1907.0,542.0,34.29,33.0,0.0,0.0,M,1,1,V40,NaN
BERGEPAINT.NS,647.0,6.96,0.17,0.83,858.0,533.0,64.92,33.0,1.0,0.0,M,1,1,V40,NaN
CAPLIPOINT.NS,736.0,3.42,-0.42,1.07,949.0,400.0,38.80,29.0,NaN,NaN,M,1,1,V40N,NaN
SUNTV.NS,446.0,-5.21,0.00,0.00,577.0,388.0,69.31,29.0,NaN,NaN,M,1,1,V40N,NaN
OFSS.NS,3597.0,20.25,0.97,1.69,4392.0,2442.0,40.77,22.0,1.0,0.0,M,1,1,V40N,NaN


In [ ]:
# top 5 from star + v40/v40n for SIP
tmp_df.sort_values(by = 'Dev%_200', ascending=True).head(5)

,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
FINEORG.NS,4473.0,-19.07,-0.72,-0.11,7196.0,2199.0,54.49,61.0,NaN,NaN,M,1,1,V40N,NaN
BAYERCROP.NS,4125.0,-9.48,-1.12,-0.10,6002.0,3970.0,92.37,46.0,NaN,NaN,M,1,1,V40N,NaN
SUNTV.NS,446.0,-5.21,0.00,0.00,577.0,388.0,69.31,29.0,NaN,NaN,M,1,1,V40N,NaN
BAJAJFINSV.NS,1439.0,-4.88,0.20,0.52,1907.0,542.0,34.29,33.0,0.0,0.0,M,1,1,V40,NaN
ANGELONE.NS,1278.0,-2.98,-0.38,1.00,1894.0,208.0,36.54,48.0,NaN,NaN,M,1,1,V40N,NaN


In [ ]:
# star + v40/v40n stocks (for breakout)

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 9


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
HINDUNILVR.NS,2652.0,3.24,0.08,0.63,2728.0,1897.0,9.15,3.0,1.0,1.0,H,1,1,V40,NaN
TITAN.NS,2742.0,7.07,0.78,1.23,2790.0,1090.0,2.82,2.0,1.0,1.0,H,1,1,V40,NaN
BAJAJ-AUTO.NS,4616.0,19.43,0.72,1.84,4644.0,2649.0,1.40,1.0,NaN,NaN,L,1,1,V40,NaN
CERA.NS,7582.0,31.97,2.58,2.13,7690.0,2229.0,1.98,1.0,1.0,1.0,H,1,1,V40N,NaN
POLYCAB.NS,3409.0,21.82,2.07,1.15,3441.0,793.0,1.21,1.0,1.0,0.0,M,1,1,V40N,NaN
AKZOINDIA.NS,2499.0,13.65,1.27,0.93,2508.0,1653.0,1.05,0.0,NaN,NaN,M,1,1,V40,NaN
EQUITASBNK.NS,84.0,45.52,3.45,1.39,84.0,33.0,0.00,0.0,1.0,1.0,M,1,1,V40N,NaN
ICICIBANK.NS,951.0,7.06,0.90,0.89,954.0,333.0,0.48,0.0,1.0,1.0,H,1,1,V40,NaN
ITC.NS,444.0,24.96,1.97,1.23,444.0,146.0,0.00,0.0,1.0,1.0,H,1,1,V40,NaN


In [ ]:
# star stocks (for upside)

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Criteria'] != 'V40') & (tmp_df['Criteria'] != 'V40N')]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 11


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
ADANIGREEN.NS,963.0,-40.20,-3.85,3.87,2970.0,453.0,79.74,208.0,-1.0,-1.0,L,1,1,NaN,NaN
ADANIENT.NS,2544.0,-11.96,-0.62,2.94,4165.0,270.0,41.62,64.0,1.0,0.0,M,1,1,NaN,NaN
LTIM.NS,5001.0,8.86,0.35,0.39,7439.0,2334.0,47.76,49.0,1.0,0.0,M,1,1,V200,NaN
EMAMILTD.NS,402.0,-5.69,-0.70,0.00,588.0,321.0,69.66,46.0,-1.0,1.0,L,1,1,V200,NaN
TATAPOWER.NS,212.0,-0.84,-0.47,0.00,287.0,49.0,31.51,35.0,1.0,0.0,M,1,1,NaN,NaN
GODREJIND.NS,485.0,10.02,0.23,1.14,644.0,354.0,54.83,33.0,0.0,1.0,L,1,1,NaN,NaN
SBICARD.NS,902.0,10.21,-0.12,1.78,1133.0,673.0,50.22,26.0,1.0,0.0,M,1,1,NaN,NaN
CIPLA.NS,951.0,-7.28,-0.19,0.66,1173.0,701.0,47.03,23.0,1.0,0.0,M,1,1,NaN,NaN
INDIANB.NS,277.0,6.65,2.31,-0.34,334.0,52.0,20.21,21.0,1.0,1.0,H,1,1,NaN,NaN


In [ ]:
# star stocks (for breakout)

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=True)

qualified stocks: 11


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
AUBANK.NS,778.0,22.36,1.26,2.15,778.0,317.0,0.00,0.0,1.0,0.0,M,1,1,NaN,NaN
EQUITASBNK.NS,84.0,45.52,3.45,1.39,84.0,33.0,0.00,0.0,1.0,1.0,M,1,1,V40N,NaN
ICICIBANK.NS,951.0,7.06,0.90,0.89,954.0,333.0,0.48,0.0,1.0,1.0,H,1,1,V40,NaN
INDUSINDBK.NS,1268.0,10.46,1.22,1.07,1272.0,484.0,0.51,0.0,1.0,1.0,H,1,1,NaN,NaN
ITC.NS,444.0,24.96,1.97,1.23,444.0,146.0,0.00,0.0,1.0,1.0,H,1,1,V40,NaN
BRITANNIA.NS,4596.0,11.67,1.02,0.64,4635.0,2945.0,2.31,1.0,1.0,1.0,H,1,1,NaN,NaN
CERA.NS,7582.0,31.97,2.58,2.13,7690.0,2229.0,1.98,1.0,1.0,1.0,H,1,1,V40N,NaN
POLYCAB.NS,3409.0,21.82,2.07,1.15,3441.0,793.0,1.21,1.0,1.0,0.0,M,1,1,V40N,NaN
TITAN.NS,2742.0,7.07,0.78,1.23,2790.0,1090.0,2.82,2.0,1.0,1.0,H,1,1,V40,NaN


In [ ]:
# stocks excluded from all time high

excluded_stocks = ['ACC.NS','AMBUJACEM.NS','CADILAHC.NS','GILLETTE.NS','NESTLEIND.NS','PGHH.NS', 'PVRINOX.NS', 'SIEMENS.NS']

tmp_df = df_prec_dev[df_prec_dev.index.isin(excluded_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 5


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
ACC.NS,1785.0,-17.48,-0.97,-0.46,2748.0,1259.0,64.67,54.0,0.0,0.0,L,0,0,NaN,NaN
PVRINOX.NS,1404.0,-14.10,-1.22,-1.28,1917.0,1348.0,90.16,37.0,1.0,-1.0,L,0,0,NaN,NaN
PGHH.NS,13440.0,-3.42,-0.17,-0.31,15778.0,9190.0,35.49,17.0,0.0,0.0,L,0,0,V40,NaN
SIEMENS.NS,3519.0,15.50,1.54,0.58,3891.0,1143.0,13.54,11.0,1.0,1.0,H,0,0,NaN,NaN
NESTLEIND.NS,21582.0,10.08,0.73,1.50,22050.0,14623.0,6.30,2.0,1.0,1.0,H,0,0,V40,NaN


In [ ]:
# stocks prospects 

excluded_stocks = ['BANDHANBNK.NS','GICRE.NS','JMFINANCIL.NS','NIACL.NS','PNB.NS','UJJIVANSFB.NS']

tmp_df = df_prec_dev[df_prec_dev.index.isin(excluded_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 0


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,


In [ ]:
# prospects with heavy upside
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[tmp_df['Upside%ATH'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 1


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
INDUSTOWER.NS,157.0,-11.5,-2.26,-0.68,303.0,137.0,87.95,93.0,1.0,1.0,H,1,0,NaN,NaN


In [ ]:
# prospects near 52W low
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[tmp_df['Discount%'] > 60]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=False)

qualified stocks: 2


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
INDUSTOWER.NS,157.0,-11.5,-2.26,-0.68,303.0,137.0,87.95,93.0,1.0,1.0,H,1,0,NaN,NaN
HDFCLIFE.NS,577.0,6.1,0.18,1.52,757.0,461.0,60.81,31.0,1.0,1.0,H,1,0,V40,NaN


In [ ]:
# prospects near 52W high
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[tmp_df['Discount%'] < 15]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=True)

qualified stocks: 16


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
EQUITASBNK.NS,84.0,45.52,3.45,1.39,84.0,33.0,0.00,0.0,1.0,1.0,M,1,1,V40N,NaN
ITC.NS,444.0,24.96,1.97,1.23,444.0,146.0,0.00,0.0,1.0,1.0,H,1,1,V40,NaN
ICICIBANK.NS,951.0,7.06,0.90,0.89,954.0,333.0,0.48,0.0,1.0,1.0,H,1,1,V40,NaN
INDUSINDBK.NS,1268.0,10.46,1.22,1.07,1272.0,484.0,0.51,0.0,1.0,1.0,H,1,1,NaN,NaN
CERA.NS,7582.0,31.97,2.58,2.13,7690.0,2229.0,1.98,1.0,1.0,1.0,H,1,1,V40N,NaN
BRITANNIA.NS,4596.0,11.67,1.02,0.64,4635.0,2945.0,2.31,1.0,1.0,1.0,H,1,1,NaN,NaN
TITAN.NS,2742.0,7.07,0.78,1.23,2790.0,1090.0,2.82,2.0,1.0,1.0,H,1,1,V40,NaN
MARUTI.NS,9400.0,7.22,0.26,0.73,9528.0,6217.0,3.87,1.0,1.0,1.0,H,1,0,NaN,NaN
BANKBARODA.NS,184.0,14.01,2.48,0.57,193.0,39.0,5.84,5.0,1.0,1.0,H,1,1,NaN,NaN


In [ ]:
# prospects in bull run
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] > 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 7


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
HDFCLIFE.NS,577.0,6.10,0.18,1.52,757.0,461.0,60.81,31.0,1.0,1.0,H,1,0,V40,NaN
NESTLEIND.NS,21582.0,10.08,0.73,1.50,22050.0,14623.0,6.30,2.0,1.0,1.0,H,0,0,V40,NaN
TITAN.NS,2742.0,7.07,0.78,1.23,2790.0,1090.0,2.82,2.0,1.0,1.0,H,1,1,V40,NaN
MARUTI.NS,9400.0,7.22,0.26,0.73,9528.0,6217.0,3.87,1.0,1.0,1.0,H,1,0,NaN,NaN
COLPAL.NS,1600.0,4.34,0.26,0.71,1719.0,1246.0,25.16,7.0,1.0,1.0,H,1,1,V40,NaN
HINDUNILVR.NS,2652.0,3.24,0.08,0.63,2728.0,1897.0,9.15,3.0,1.0,1.0,H,1,1,V40,NaN
HAVELLS.NS,1289.0,4.64,0.08,0.49,1461.0,606.0,20.12,13.0,1.0,1.0,H,1,0,V40,NaN


In [ ]:
# prospects about to turn around
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] < 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 1


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
INDUSTOWER.NS,157.0,-11.5,-2.26,-0.68,303.0,137.0,87.95,93.0,1.0,1.0,H,1,0,NaN,NaN


In [ ]:
# cross-check the features
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] != 1) | (tmp_df['BizDurability'] != 1)]
tmp_df = tmp_df[tmp_df['Upside%ATH'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 47


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
ADANITRANS.NS,850.0,-63.20,-4.85,0.42,4105.0,235.0,84.11,383.0,1.0,-1.0,L,0,0,NaN,NaN
PEL.NS,771.0,-15.15,-5.72,0.00,2866.0,635.0,93.90,272.0,-1.0,0.0,L,1,0,NaN,NaN
ADANIGREEN.NS,963.0,-40.20,-3.85,3.87,2970.0,453.0,79.74,208.0,-1.0,-1.0,L,1,1,NaN,NaN
IBREALEST.NS,68.0,-6.18,0.00,0.00,191.0,47.0,85.42,181.0,-1.0,-1.0,L,0,0,NaN,NaN
JCHAC.NS,1040.0,-15.04,-2.04,-1.36,2837.0,1017.0,98.74,173.0,NaN,NaN,M,1,0,V40N,NaN
LALPATHLAB.NS,2026.0,-8.58,-0.36,0.63,4186.0,1748.0,88.60,107.0,NaN,NaN,M,1,0,V40N,NaN
BIOCON.NS,239.0,-7.90,-1.15,0.88,483.0,199.0,85.92,102.0,1.0,0.0,M,1,0,NaN,NaN
SFL.NS,1020.0,-19.40,-1.50,-1.24,1972.0,623.0,70.57,93.0,NaN,NaN,M,1,0,V40N,NaN
ZEEL.NS,191.0,-18.46,-0.85,-1.01,367.0,164.0,86.70,92.0,-1.0,1.0,L,1,0,NaN,NaN


In [ ]:
# analyse individual stock
stock_id = 'PVRINOX.NS'
df_prec_dev[df_prec_dev.index == stock_id]

,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
PVRINOX.NS,1404.0,-14.1,-1.22,-1.28,1917.0,1348.0,90.16,37.0,1.0,-1.0,L,0,0,NaN,NaN


In [ ]:
# save the output
#df_prec_dev = df_prec_dev.reset_index()
#df_prec_dev.to_csv('/content/drive/My Drive/data/stocks/Nifty100-Upside-Analysis-ATH.csv', index=False)

In [ ]:
# nifty in last 5 weeks
#stock_ids = ['ICICIGI','DABUR','MOTILALOFS','JPPOWER','EQUITAS','GLAND','JMFINANCIL','LUXIND','UJJIVANSFB','VAIBHAVGBL','VALIANTORG']
stock_prec_dev('BHARTIARTL.NS')[-21::5]

,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH
Date,,,,,,,,,
2023-04-27 00:00:00+05:30,786.0,BHARTIARTL.NS,2.01,0.65,-0.26,851.0,396.0,14.29,8.0
2023-05-05 00:00:00+05:30,787.0,BHARTIARTL.NS,1.72,0.78,0.26,851.0,396.0,14.07,8.0
2023-05-12 00:00:00+05:30,792.0,BHARTIARTL.NS,1.92,0.77,0.26,851.0,396.0,12.97,7.0
2023-05-19 00:00:00+05:30,806.0,BHARTIARTL.NS,3.30,0.77,0.52,851.0,396.0,9.89,6.0
2023-05-26 00:00:00+05:30,818.0,BHARTIARTL.NS,4.46,0.77,0.52,851.0,396.0,7.25,4.0
